<a href="https://colab.research.google.com/github/Meisam-sh/Machine_Learning_Python_With_Meisam_Shabanpoor/blob/main/Pytorch_Tensor_movies_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np

# فرض کنید داده‌های تعاملات کاربران و فیلم‌ها در این ماتریس قرار دارد
user_item_matrix = np.array([[5, 3, 0, 1],
                            [4, 0, 0, 1],
                            [1, 1, 2, 5],
                            [1, 0, 0, 4],
                            [0, 1, 5, 4]])


In [ ]:

# تبدیل ماتریس به PyTorch tensor
user_item_tensor = torch.tensor(user_item_matrix, dtype=torch.float32)

# تعریف مدل
class RecommenderModel(nn.Module):
    def __init__(self, num_users, num_items, embedding_size):
        super(RecommenderModel, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embedding_size)
        self.item_embeddings = nn.Embedding(num_items, embedding_size)
        self.fc = nn.Linear(embedding_size * 2, 1)

    def forward(self, user_ids, item_ids):
        user_emb = self.user_embeddings(user_ids)
        item_emb = self.item_embeddings(item_ids)
        concat = torch.cat([user_emb, item_emb], dim=1)
        output = self.fc(concat)
        return output.squeeze()

# تعداد کاربران و آیتم‌ها
num_users = user_item_tensor.shape[0]
num_items = user_item_tensor.shape[1]

# ایجاد مدل
model = RecommenderModel(num_users, num_items, embedding_size=50)

# تعریف تابع هزینه و بهینه‌ساز
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# آموزش مدل
num_epochs = 100
for epoch in range(num_epochs):
    # محاسبه خروجی مدل برای همه کاربران و آیتم‌ها
    outputs = model(torch.arange(num_users), torch.arange(num_items))

    # محاسبه تابع هزینه و به‌روزرسانی پارامترهای مدل
    loss = criterion(outputs, user_item_tensor.flatten())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# پیشنهاد آیتم‌ها برای کاربر با شناسه 2
user_id = 2
predicted_ratings = model(torch.tensor([user_id]), torch.arange(num_items))
top_recommendations = torch.argsort(predicted_ratings, descending=True)
print(f"Top recommendations for user {user_id}: {top_recommendations.tolist()}")